In [3]:
import os, json
from datetime import datetime, timedelta
from pymongo import MongoClient, errors
from dotenv import load_dotenv

# -----------------------------
# 1) Input: previous document
# -----------------------------
previous = {
    "id": "9f3a27e6-1b52-4e0b-bc6e-62c1f2a7c8b9",
    "previous_form_qr_code": "20251011",
    "current_form_qr_code": "20251012",
    "date": "2025-10-12",
    "location": "JEF Gas Station – Sikatuna Branch",
    "tin": "123-456-789",
    "branch": "JEF Biosciences Fuel Division",
    "items": [
        {"barcode": "4809992000101", "type": "fuel", "name": "Premium Gasoline", "price": 69.50, "unit": "liter", "previous_quantity": 520, "addstock": 300, "sold": 250, "current_quantity": 570},
        {"barcode": "4809992000102", "type": "fuel", "name": "Regular Gasoline", "price": 64.75, "unit": "liter", "previous_quantity": 680, "addstock": 400, "sold": 320, "current_quantity": 760},
        {"barcode": "4809992000103", "type": "fuel", "name": "Premium Diesel", "price": 61.20, "unit": "liter", "previous_quantity": 450, "addstock": 350, "sold": 280, "current_quantity": 520},
        {"barcode": "4809992000201", "type": "lubricant", "name": "Engine Oil SAE 40", "price": 250.00, "unit": "liter", "previous_quantity": 60, "addstock": 40, "sold": 25, "current_quantity": 75},
        {"barcode": "4809992000202", "type": "lubricant", "name": "Automatic Transmission Fluid (ATF)", "price": 310.00, "unit": "liter", "previous_quantity": 50, "addstock": 30, "sold": 20, "current_quantity": 60},
        {"barcode": "4809992000203", "type": "lubricant", "name": "Gear Oil EP 90", "price": 285.00, "unit": "liter", "previous_quantity": 45, "addstock": 25, "sold": 15, "current_quantity": 55},
        {"barcode": "4809992000204", "type": "lubricant", "name": "Hydraulic Oil ISO 68", "price": 295.00, "unit": "liter", "previous_quantity": 35, "addstock": 25, "sold": 10, "current_quantity": 50},
        {"barcode": "4809992000205", "type": "lubricant", "name": "2T Motorcycle Oil", "price": 180.00, "unit": "liter", "previous_quantity": 80, "addstock": 60, "sold": 45, "current_quantity": 95}
    ],
    "cashier": "Liam Santos",
    "cashier_employee_number": "50321",
    "recorder": "Ella Reyes",
    "recorder_employee_number": "50325"
}

# -----------------------------
# 2) Create next-day empty form
# -----------------------------
def create_empty_form(previous_doc):
    today = datetime.strptime(previous_doc["date"], "%Y-%m-%d") + timedelta(days=1)
    new_doc = {
        # let Mongo generate _id; omit app-level "id"
        "previous_form_qr_code": previous_doc["current_form_qr_code"],
        "current_form_qr_code": today.strftime("%Y%m%d"),
        "date": today.strftime("%Y-%m-%d"),
        "location": previous_doc["location"],
        "tin": previous_doc["tin"],
        "branch": previous_doc["branch"],
        "items": [
            {
                "barcode": it["barcode"],
                "type": it["type"],
                "name": it["name"],
                "price": it["price"],
                "unit": it["unit"],
                "previous_quantity": it["current_quantity"],
                # omit optional-entry fields when empty for cleaner storage
                # (will be added later when user inputs values)
            }
            for it in previous_doc["items"]
        ],
        # omit empty optional top-level fields as well
    }
    return new_doc

new_form = create_empty_form(previous)

# -----------------------------
# 3) Save to MongoDB
# -----------------------------
load_dotenv()
uri = os.getenv("MONGODB_URI")
database = os.getenv("MONGODB_DATABASE")
collection = "inventories"

client = MongoClient(uri, retryWrites=True)
db = client[database]
col = db[collection]

# Ensure uniqueness by QR code (idempotent writes across runs)
col.create_index("current_form_qr_code", unique=True)

try:
    # Upsert by current_form_qr_code: insert if new, keep existing if already present
    res = col.update_one(
        {"current_form_qr_code": new_form["current_form_qr_code"]},
        {"$setOnInsert": new_form},
        upsert=True,
    )
    if res.upserted_id is not None:
        print("✅ Inserted new form with _id:", res.upserted_id)
    else:
        print("⚠️ Form already exists for current_form_qr_code:", new_form["current_form_qr_code"])

    # Read back and show a compact view
    saved = col.find_one({"current_form_qr_code": new_form["current_form_qr_code"]}, {"_id": 0})
    print(json.dumps(saved, indent=2, ensure_ascii=False))
except errors.DuplicateKeyError:
    print("⚠️ Duplicate current_form_qr_code; document already exists.")
finally:
    client.close()


✅ Inserted new form with _id: 68ec7a33a268c4061366ba56
{
  "current_form_qr_code": "20251013",
  "branch": "JEF Biosciences Fuel Division",
  "date": "2025-10-13",
  "items": [
    {
      "barcode": "4809992000101",
      "type": "fuel",
      "name": "Premium Gasoline",
      "price": 69.5,
      "unit": "liter",
      "previous_quantity": 570
    },
    {
      "barcode": "4809992000102",
      "type": "fuel",
      "name": "Regular Gasoline",
      "price": 64.75,
      "unit": "liter",
      "previous_quantity": 760
    },
    {
      "barcode": "4809992000103",
      "type": "fuel",
      "name": "Premium Diesel",
      "price": 61.2,
      "unit": "liter",
      "previous_quantity": 520
    },
    {
      "barcode": "4809992000201",
      "type": "lubricant",
      "name": "Engine Oil SAE 40",
      "price": 250.0,
      "unit": "liter",
      "previous_quantity": 75
    },
    {
      "barcode": "4809992000202",
      "type": "lubricant",
      "name": "Automatic Transmission F